[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VitcSS/Genre_Reveal_Party/blob/master/AST.ipynb)

## Setup
### Install the packages:

In [26]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if RunningInCOLAB:
    ! pip install -q git+https://github.com/huggingface/transformers@a2f55a65cd0eb3bde0db4d5102a824ec96c7d7e9
    ! pip3 install torch torchaudio torchvision torchtext torchdata
    ! pip install -q datasets
    ! pip install -q transformers
    ! pip install -q evaluate
    ! pip install -q numpy
    ! pip install -q huggingface_hub
    ! pip install transformers[torch]
    ! pip install accelerate -U
else:
    ! pip install -q -r requirements.txt

### Import libraries :

In [27]:
from datasets import load_dataset
from datasets import Audio
from huggingface_hub import notebook_login
from transformers import ASTFeatureExtractor
from transformers import ASTForAudioClassification
from transformers import Trainer
from transformers import TrainingArguments
import evaluate
import numpy as np

##  Model Information

In [28]:
model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"

## Data Preparation
### Load Dataset

In [29]:
gtzan = load_dataset("marsyas/gtzan", "all")

In [30]:
# Train Test Split
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)

In [31]:
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

### Feature Extraction

#### Get Feature Extractor

In [32]:
feature_extractor = ASTFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
sampling_rate = feature_extractor.sampling_rate

### Resample Audio Data

In [33]:
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

### Extract Features

In [34]:
def preprocess_function(examples, max_duration = 30.0):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [35]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

### Data Labelling

In [36]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

## Instantiate the model

In [39]:
# total number of genres 
num_labels = len(id2label)

model = ASTForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

: 

### Model Training Parameters

In [38]:
model_name = 'GRP'
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16= True , # Set to true if using CUDA
    push_to_hub=False,
)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or NPU devices or certain XPU devices (with IPEX).

### Training Evaluation Metrics

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

### Instantiate Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

### Train the Model

In [ ]:
trainer.train()